<a href="https://colab.research.google.com/github/theelderemo/HeartLib-Google-Colab/blob/main/HeartLib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup Environment
import os

!sudo apt-get update && sudo apt-get install -y ffmpeg

if not os.path.exists("heartlib"):
    !git clone https://github.com/HeartMuLa/heartlib.git

%cd heartlib

!pip install -e .
!pip install -u "huggingface_hub[cli]"
!pip install flash-attn --no-build-isolation
!pip install accelerate

In [ ]:
# @title Download Checkpoints
import os

if os.path.basename(os.getcwd()) != "heartlib":
    %cd heartlib

os.makedirs('./ckpt', exist_ok=True)

!huggingface-cli download --local-dir './ckpt' 'HeartMuLa/HeartMuLaGen'
!huggingface-cli download --local-dir './ckpt/HeartMuLa-oss-3B' 'HeartMuLa/HeartMuLa-oss-3B'
!huggingface-cli download --local-dir './ckpt/HeartCodec-oss' 'HeartMuLa/HeartCodec-oss'
!huggingface-cli download --local-dir './ckpt/HeartTranscriptor-oss' 'HeartMuLa/HeartTranscriptor-oss'

In [ ]:
# @title Run Music Generation
# No Touchy
import os
from IPython.display import Audio, display
import torch

torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision('high')

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'

# -----------------------------------------------------------------------------
# 1. SETUP YOUR LYRICS AND TAGS (you can touch this lol)
# -----------------------------------------------------------------------------
my_lyrics = """
[Verse]
The sun creeps in across the floor
I hear the traffic outside the door
The coffee pot begins to hiss
It is another morning just like this

[Chorus]
Every day the light returns
Every day the fire burns
"""

my_tags = "piano,happy,pop"
# -----------------------------------------------------------------------------
# Do Not Touch This Code Below, lol.

if os.path.basename(os.getcwd()) != "heartlib":
    if os.path.exists("heartlib"):
        %cd heartlib
    else:
        raise FileNotFoundError("Repo not found. Please run Block 1 (Setup) first.")

!python ./examples/run_music_generation.py \
    --model_path=./ckpt \
    --version="3B" \
    --lyrics="$my_lyrics" \
    --tags="$my_tags" \
    --save_path="./assets/output.mp3"

if os.path.exists("./assets/output.mp3"):
    print("✓ Generation complete!")
    display(Audio("./assets/output.mp3"))
else:
    print("✗ Generation failed. Check logs above.")
